*Manual*        
* Register https://urs.earthdata.nasa.gov/users/new           
* Get Map Ket  normal user 1000 transaccions/10 minuts  (52536e8e34cf07bde80f3177cdcd7771)
* 

In [1]:
import requests
import pandas as pd
import numpy as np
from io import StringIO
from datetime import datetime, timedelta
import webbrowser


import folium



In [2]:
# Texas cordenates 
min_lon, min_lat, max_lon, max_lat = "-106.6","25.8","-93.5","36.5"
cordate_list = ",".join([min_lon, min_lat, max_lon, max_lat])
map_key = "52536e8e34cf07bde80f3177cdcd7771"
url = f'https://firms.modaps.eosdis.nasa.gov/api/area/csv/{map_key}/'
# Fire season is summer  june_1 to september 28  june(31), july(30) august(31), september(30)
start_date = datetime(2022, 6, 1)
days = 50
iterations = int(days/10)

In [3]:
def read_excel_file(response, iteration,requests_day):
    try:
        df = pd.read_csv(StringIO(response.text))
        print(f"Response {iteration} -- {requests_day} read correctly with {len(df)} fires detected")
        return df
    except FileNotFoundError:
        print(f"The file '{response}' was not found.")
        return None
    except pd.errors.ParserError:
        print("Error while parsing the file.")
        return None
def save_transformed_data(df, output_path):
    if df is not None:
        try:
            df.to_csv(output_path, index=False)
            print("Data Frame saved correctly ")

        except Exception as e:
            print(f"An error occurred while saving the transformed data: {str(e)}")

def dowload_and_tranform_data(cordate_list,map_key,start_date,iterations):
    data_frames = []
    requests_day = start_date
    for i in range(iterations):
        url_test = f"https://firms.modaps.eosdis.nasa.gov/api/area/csv/{map_key}/LANDSAT_NRT/{cordate_list}/10/{requests_day.strftime('%Y-%m-%d')}"
        response = requests.get(url_test)
        if response.status_code == 200:
            df = read_excel_file(response, i,requests_day)
            data_frames.append(df)
            
        else:
            print(f"Error en la solicitud: {response.status_code}")
        requests_day += timedelta(days=10)
    final_df = pd.concat(data_frames, ignore_index=True)
    return final_df

final_df = dowload_and_tranform_data(cordate_list,map_key,start_date,iterations)
final_df = final_df.drop_duplicates()
save_transformed_data(final_df, "./final_df.csv")

Response 0 -- 2022-06-01 00:00:00 read correctly with 0 fires detected
Response 1 -- 2022-06-11 00:00:00 read correctly with 155 fires detected
Response 2 -- 2022-06-21 00:00:00 read correctly with 2481 fires detected
Response 3 -- 2022-07-01 00:00:00 read correctly with 2039 fires detected
Response 4 -- 2022-07-11 00:00:00 read correctly with 703 fires detected
Data Frame saved correctly 


C:\Users\fabri\AppData\Local\Temp\ipykernel_19848\4050060035.py:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat(data_frames, ignore_index=True)


In [4]:

# Convert acq_date and acq_time into a single datetime object
final_df['acq_datetime'] = pd.to_datetime(final_df['acq_date'] + ' ' + final_df['acq_time'].astype(str).str.zfill(4), format='%Y-%m-%d %H%M')

# Sort the data by datetime and location for easier comparison
fire_data_sorted = final_df.sort_values(by=['latitude', 'longitude', 'acq_datetime'])

# Define a function to calculate Euclidean distance between two points
def calculate_distance(lat1, lon1, lat2, lon2):
    # Approximate radius of earth in km
    R = 6371.0
    dlat = np.radians(lat2 - lat1)
    dlon = np.radians(lon2 - lon1)
    a = np.sin(dlat / 2)**2 + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance = R * c * 1000 # Convert to meters
    return distance

# Add columns for lat/lon difference and time difference to the sorted data
fire_data_sorted['lat_diff'] = fire_data_sorted['latitude'].diff().abs()
fire_data_sorted['lon_diff'] = fire_data_sorted['longitude'].diff().abs()
fire_data_sorted['time_diff'] = fire_data_sorted['acq_datetime'].diff().abs()

# Calculate spatial distance between consecutive observations
fire_data_sorted['spatial_distance'] = fire_data_sorted.apply(lambda row: calculate_distance(row['latitude'], row['longitude'], 
                                                                                              row['latitude'] - row['lat_diff'], 
                                                                                              row['longitude'] - row['lon_diff']), axis=1)

# Show the updated dataframe
fire_data_sorted.head()


,latitude,longitude,path,row,scan,track,acq_date,acq_time,satellite,confidence,daynight,acq_datetime,lat_diff,lon_diff,time_diff,spatial_distance
3248,25.805767,-100.356592,28,42,4354,4598,2022-07-06,1710,L9,H,D,2022-07-06 17:10:00,NaN,NaN,NaT,NaN
129,25.805770,-100.356293,28,42,4395,4598,2022-06-20,1710,L9,H,D,2022-06-20 17:10:00,0.000003,0.000299,16 days,29.933555
5110,25.806035,-100.356894,28,42,4253,4597,2022-07-14,1710,L8,H,D,2022-07-14 17:10:00,0.000265,0.000601,24 days,66.992146
130,25.806038,-100.356595,28,42,4394,4597,2022-06-20,1710,L9,H,D,2022-06-20 17:10:00,0.000003,0.000299,24 days,29.933487
3249,25.806038,-100.356595,28,42,4354,4597,2022-07-06,1710,L9,H,D,2022-07-06 17:10:00,0.000000,0.000000,16 days,0.000000


In [5]:
# Define thresholds for clustering
spatial_threshold = 60  # 60 meters
temporal_threshold = pd.Timedelta('24 hours')  # 24 hours

# Initialize fire cluster id
fire_data_sorted['fire_cluster'] = 0
current_cluster_id = 1

# Iterate over the sorted data to assign fire cluster ids
for i in range(1, len(fire_data_sorted)):
    if (fire_data_sorted.iloc[i]['spatial_distance'] <= spatial_threshold) and \
       (fire_data_sorted.iloc[i]['time_diff'] <= temporal_threshold):
        fire_data_sorted.at[fire_data_sorted.index[i], 'fire_cluster'] = current_cluster_id
    else:
        current_cluster_id += 1
        fire_data_sorted.at[fire_data_sorted.index[i], 'fire_cluster'] = current_cluster_id

# Display some of the data to verify the clustering
fire_data_sorted[['latitude', 'longitude', 'acq_datetime', 'spatial_distance', 'time_diff', 'fire_cluster']].head(10)


,latitude,longitude,acq_datetime,spatial_distance,time_diff,fire_cluster
3248,25.805767,-100.356592,2022-07-06 17:10:00,NaN,NaT,0
129,25.805770,-100.356293,2022-06-20 17:10:00,29.933555,16 days,2
5110,25.806035,-100.356894,2022-07-14 17:10:00,66.992146,24 days,3
130,25.806038,-100.356595,2022-06-20 17:10:00,29.933487,24 days,4
3249,25.806038,-100.356595,2022-07-06 17:10:00,0.000000,16 days,5
5111,25.806038,-100.356595,2022-07-14 17:10:00,0.000000,8 days,6
131,25.806041,-100.356296,2022-06-20 17:10:00,29.933486,24 days,7
1905,25.806041,-100.356296,2022-06-28 17:10:00,0.000000,8 days,8
3250,25.806041,-100.356296,2022-07-06 17:10:00,0.000000,8 days,9
5112,25.806041,-100.356296,2022-07-14 17:10:00,0.000000,8 days,10


In [6]:
# Group by fire_cluster to calculate properties of each fire event
fire_events = fire_data_sorted.groupby('fire_cluster').agg(
    start_time=pd.NamedAgg(column='acq_datetime', aggfunc='min'),
    end_time=pd.NamedAgg(column='acq_datetime', aggfunc='max'),
    min_latitude=pd.NamedAgg(column='latitude', aggfunc='min'),
    max_latitude=pd.NamedAgg(column='latitude', aggfunc='max'),
    min_longitude=pd.NamedAgg(column='longitude', aggfunc='min'),
    max_longitude=pd.NamedAgg(column='longitude', aggfunc='max'),
    detections=pd.NamedAgg(column='latitude', aggfunc='count')
)

# Calculate duration and area for each fire event
fire_events['duration'] = fire_events['end_time'] - fire_events['start_time']
fire_events['area'] = np.pi * (30 ** 2) * fire_events['detections']  # Area per detection assuming 30m radius

# Sort fire events by area
top_fires = fire_events.sort_values(by='area', ascending=False).head(10)
top_fires["mean_latitud"] = (top_fires["max_latitude"]  + top_fires["max_latitude"]) /2
top_fires["mean_longitude"] = (top_fires["min_longitude"]  + top_fires["max_longitude"]) /2

# Display the top 10 fire events
top_fires[['start_time', 'end_time', 'duration', 'detections', 'area', 'min_latitude','max_latitude', 'min_longitude','max_longitude',"mean_longitude","mean_latitud"]]


,start_time,end_time,duration,detections,area,min_latitude,max_latitude,min_longitude,max_longitude,mean_longitude,mean_latitud
fire_cluster,,,,,,,,,,,
625,2022-07-08 16:57:00,2022-07-08 16:57:00,0 days 00:00:00,29,81995.568259,29.021130,29.024193,-96.042581,-96.038643,-96.040612,29.024193
1954,2022-06-30 16:55:00,2022-06-30 16:55:00,0 days 00:00:00,28,79168.134870,35.431070,35.432710,-95.250272,-95.249296,-95.249784,35.432710
1637,2022-06-30 16:55:00,2022-06-30 16:56:00,0 days 00:01:00,28,79168.134870,33.781653,33.781707,-94.577689,-94.573478,-94.575583,33.781707
1642,2022-06-30 16:55:00,2022-06-30 16:56:00,0 days 00:01:00,26,73513.268094,33.783006,33.783055,-94.577714,-94.573827,-94.575771,33.783055
1641,2022-06-30 16:55:00,2022-06-30 16:56:00,0 days 00:01:00,26,73513.268094,33.782735,33.782785,-94.577709,-94.573822,-94.575766,33.782785
1640,2022-06-30 16:55:00,2022-06-30 16:56:00,0 days 00:01:00,26,73513.268094,33.782465,33.782514,-94.577704,-94.573817,-94.575761,33.782514
1638,2022-06-30 16:55:00,2022-06-30 16:56:00,0 days 00:01:00,26,73513.268094,33.781924,33.781974,-94.577694,-94.573807,-94.575750,33.781974
1571,2022-06-27 17:14:00,2022-06-27 17:14:00,0 days 00:00:00,26,73513.268094,33.365948,33.367854,-99.327374,-99.322865,-99.325119,33.367854
1639,2022-06-30 16:55:00,2022-06-30 16:56:00,0 days 00:01:00,24,67858.401318,33.782199,33.782244,-94.577375,-94.573812,-94.575593,33.782244


In [7]:
def radius_from_area(area):
    radius = np.sqrt(area / np.pi)
    return radius

def graph_fire_area(row):
    # map cordenates
    latitud = row["mean_latitud"]  
    longitud = row["mean_longitude"]

    # Crea un mapa centrado en las coordenadas especificadas
    mapa = folium.Map(location=[latitud, longitud], zoom_start=15)
    radius = radius_from_area(row["area"])
    # Agrega un círculo al mapa para representar el fuego
    folium.Circle(location=[latitud, longitud], radius=radius, color='red', fill=True, fill_opacity=0.3).add_to(mapa)
    # Agrega un círculo al mapa para representar area 1 
    folium.Circle(location=[latitud, longitud], radius=radius*2, color='yellow', fill=True, fill_opacity=0.3).add_to(mapa)

    # Agrega un círculo al mapa para representar area 2
    folium.Circle(location=[latitud, longitud], radius=radius*4, color='blue', fill=True, fill_opacity=0.3).add_to(mapa)

    # Agrega un marcador en el centro del mapa
    folium.Marker([latitud, longitud], icon=folium.Icon(color='red')).add_to(mapa)

    # Guarda el mapa como un archivo HTML
    mapa.save("mapa.html")

    # Abre el mapa en tu navegador web predeterminado
    webbrowser.open("mapa.html")

graph_fire_area(top_fires.iloc[0])
